In [46]:
import pandas as pd
from sklearn.model_selection import train_test_split
import time
import numpy as np


timers = []
timers_import_start = time.time() 
# Trainer (has labels)
#trainer_df = pd.read_csv('../datasets/cleaned_SOLIDtest6K_trainer.tsv', sep="\t")
olid_df = pd.read_csv('../datasets/cleaned_OLID.tsv', sep="\t")
learner_tweets_df = pd.read_csv('../datasets/cleaned_SOLID9M_learner.tsv', sep="\t")
tr_tweets_df = pd.read_csv('../datasets/cleaned_tr_offenseval.tsv', sep="\t")
hso_df = pd.read_csv('../datasets/cleaned_hatespeech_offensive_train.tsv', sep="\t")


timers_import_end = time.time()
print(timers_import_end - timers_import_start)

11.528602361679077


In [35]:
## tr_offenseval_dataset

import numpy as np
import torch
from torch.utils.data import DataLoader, Dataset

tr_tweets = tr_tweets_df['tweet'].values
tr_labels = tr_tweets_df['label'].values


class TweetDatasetTrOffensEval(Dataset):
    def __init__(self, texts, labels=None):
        self.texts = texts
        self.labels = labels

    def __len__(self):
        return len(self.texts)

    def __getitem__(self, idx):
        text = str(self.texts[idx])
        if self.labels is not None:
            label = self.labels[idx]
            return {
                'text': text,
                'label': label
            }
        else:
            return {
                'text': text
            }
        
     
# trainer_dataset = TweetDataset(trainer_tweets, trainer_labels)
troffenseval_train_dataset = TweetDatasetTrOffensEval(tr_tweets, tr_labels)

# trainer_loader = DataLoader(trainer_dataset, batch_size=12, shuffle=True)
troffenseval_train_loader = DataLoader(troffenseval_train_dataset, batch_size=6, shuffle=False)


In [36]:
## olid_dataset
#olid_df['subtask_a'] = olid_df['subtask_a'].map({'OFF': 1, 'NOT': 0})

olid_df['labels'] = olid_df["label"]
#olid_df.drop(["subtask_a","subtask_b","subtask_c"], axis=1, inplace=True)


olid_final_df = olid_df
olid_final_df = olid_final_df.sample(frac=1, random_state=42)

semi_tweets = olid_final_df['tweet'].values
semi_labels = olid_final_df['label'].values


class TweetDatasetOlid(Dataset):
    def __init__(self, texts, labels=None):
        self.texts = texts
        self.labels = labels

    def __len__(self):
        return len(self.texts)

    def __getitem__(self, idx):
        text = str(self.texts[idx])
        if self.labels is not None:
            label = self.labels[idx]
            return {
                'text': text,
                'label': label
            }
        else:
            return {
                'text': text
            }
        
# trainer_dataset = TweetDataset(trainer_tweets, trainer_labels)
olid_learner_dataset = TweetDatasetOlid(semi_tweets, semi_labels)

# trainer_loader = DataLoader(trainer_dataset, batch_size=12, shuffle=True)
olid_learner_loader = DataLoader(olid_learner_dataset, batch_size=6, shuffle=False)


In [48]:
## HSO dataset

hso_tweets = hso_df['tweet'].values
hso_labels = hso_df['label'].values


class TweetDatasetHSO(Dataset):
    def __init__(self, texts, labels=None):
        self.texts = texts
        self.labels = labels

    def __len__(self):
        return len(self.texts)

    def __getitem__(self, idx):
        text = str(self.texts[idx])
        if self.labels is not None:
            label = self.labels[idx]
            return {
                'text': text,
                'label': label
            }
        else:
            return {
                'text': text
            }
        
     
# trainer_dataset = TweetDataset(trainer_tweets, trainer_labels)
hso_train_dataset = TweetDatasetHSO(tr_tweets, tr_labels)

# trainer_loader = DataLoader(trainer_dataset, batch_size=12, shuffle=True)
hso_train_loader = DataLoader(troffenseval_train_dataset, batch_size=6, shuffle=False)


In [37]:
## solid 
import numpy as np
# trainer_labels = trainer_df['label'].values
# trainer_tweets = trainer_df['tweet'].values
learner_tweets_df['labels'] = learner_tweets_df['average'].apply(lambda x: 1 if x >= 0.8 else 0) # threshold the average values

sample_size = 80000
positive_ratio = 0.75

# Select the most confident positive values
semi_tweets_pos_df = learner_tweets_df[learner_tweets_df['average'] > 0.8].sample(n=np.floor(sample_size*positive_ratio).astype(int), random_state=1)

# Select the most confident negative values
semi_tweets_neg_df = learner_tweets_df[learner_tweets_df['average'] < 0.2].sample(n=np.floor(sample_size*(1-positive_ratio)).astype(int), random_state=1)

semi_tweets_df = pd.concat([semi_tweets_pos_df, semi_tweets_neg_df])
semi_tweets_df = semi_tweets_df.sample(frac=1, random_state=42)

semi_tweets = semi_tweets_df['text'].values
semi_labels = semi_tweets_df['labels'].values

import torch
from torch.utils.data import DataLoader, Dataset


class TweetDatasetSolid(Dataset):
    def __init__(self, texts, labels=None):
        self.texts = texts
        self.labels = labels

    def __len__(self):
        return len(self.texts)

    def __getitem__(self, idx):
        text = str(self.texts[idx])
        if self.labels is not None:
            label = self.labels[idx]
            return {
                'text': text,
                'label': label
            }
        else:
            return {
                'text': text
            }
        
# trainer_dataset = TweetDataset(trainer_tweets, trainer_labels)
solid_learner_dataset = TweetDatasetSolid(semi_tweets, semi_labels)

# trainer_loader = DataLoader(trainer_dataset, batch_size=12, shuffle=True)
solid_learner_loader = DataLoader(solid_learner_dataset, batch_size=6, shuffle=False)

In [38]:
import torch
import torch.nn as nn
from torch import optim
from tqdm import tqdm
from transformers import DistilBertTokenizer, DistilBertModel, DistilBertForSequenceClassification

# Number of epochs

base_model_name = 'distilbert-base-uncased'
full_model_name = 'distilbert/distilbert-base-uncased'

tokenizer = DistilBertTokenizer.from_pretrained(full_model_name)

# Initialize the model
baseModel = DistilBertForSequenceClassification.from_pretrained(full_model_name, num_labels=2)


# Set up the optimizer and loss function
optimizer = optim.AdamW(baseModel.parameters(), lr=1e-5)
criterion = torch.nn.CrossEntropyLoss()

# Check if CUDA is available and set the device accordingly
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')


C:\Users\mdoga\AppData\Local\Programs\Python\Python312\Lib\site-packages\huggingface_hub\file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert/distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [39]:
## olid training
olid_model = baseModel
olid_model.to(device)
olid_model.train()
num_epochs = 1
optimizer = optim.AdamW(olid_model.parameters(), lr=1e-5)

timer_olid_start = time.time()
for epoch in range(num_epochs):
    # trainer_dataloader_iterator = iter(trainer_loader)
    dataloader_iter = iter(olid_learner_loader)
    num_batches = len(olid_learner_loader)
    for _ in tqdm(range(num_batches)):
        # Train on labeled data
        batch = next(dataloader_iter, None)
        if batch is not None:
            inputs = tokenizer(batch['text'], padding=True, truncation=True, return_tensors="pt").to(device)
            labels = torch.tensor(batch['label']).clone().detach().to(device)
            outputs = olid_model(**inputs)
            logits = outputs.logits
    
            loss = criterion(logits, labels)
            loss.backward()
            optimizer.step()
            optimizer.zero_grad()
olid_model.to('cpu')
timer_olid_end = time.time()

  0%|                                                                                         | 0/2207 [00:00<?, ?it/s]C:\Users\mdoga\AppData\Local\Temp\ipykernel_59176\3080580362.py:18: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  labels = torch.tensor(batch['label']).clone().detach().to(device)
100%|██████████████████████████████████████████████████████████████████████████████| 2207/2207 [01:16<00:00, 28.69it/s]


In [40]:
olid_model.save_pretrained(f'models/{base_model_name}_olid')

In [41]:

## solid training

olidSolid_model = olid_model
olidSolid_model.to(device)

olidSolid_model.train()
num_epochs = 1
optimizer = optim.AdamW(olidSolid_model.parameters(), lr=1e-5)


timer_olidsolid_start = time.time()
for epoch in range(num_epochs):
    # trainer_dataloader_iterator = iter(trainer_loader)
    dataloader_iter = iter(solid_learner_loader)
    num_batches = len(solid_learner_loader)
    for _ in tqdm(range(num_batches)):
        # Train on labeled data
        batch = next(dataloader_iter, None)
        if batch is not None:
            inputs = tokenizer(batch['text'], padding=True, truncation=True, return_tensors="pt").to(device)
            labels = torch.tensor(batch['label']).clone().detach().to(device)
            outputs = olidSolid_model(**inputs)
            logits = outputs.logits
    
            loss = criterion(logits, labels)
            loss.backward()
            optimizer.step()
            optimizer.zero_grad()
olidSolid_model.to("cpu")

timer_olidsolid_end = time.time()

olidSolid_model.save_pretrained(f'models/{base_model_name}_olid_solid')

  0%|                                                                                        | 0/13334 [00:00<?, ?it/s]C:\Users\mdoga\AppData\Local\Temp\ipykernel_59176\2126588602.py:21: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  labels = torch.tensor(batch['label']).clone().detach().to(device)
100%|████████████████████████████████████████████████████████████████████████████| 13334/13334 [06:40<00:00, 33.31it/s]


In [42]:
## pure solid

solid_model = baseModel
solid_model.to(device)

solid_model.train()

num_epochs = 1
optimizer = optim.AdamW(solid_model.parameters(), lr=1e-5)


timer_solid_start = time.time()
for epoch in range(num_epochs):
    # trainer_dataloader_iterator = iter(trainer_loader)
    dataloader_iter = iter(solid_learner_loader)
    num_batches = len(solid_learner_loader)
    for _ in tqdm(range(num_batches)):
        # Train on labeled data
        batch = next(dataloader_iter, None)
        if batch is not None:
            inputs = tokenizer(batch['text'], padding=True, truncation=True, return_tensors="pt").to(device)
            labels = torch.tensor(batch['label']).clone().detach().to(device)
            outputs = solid_model(**inputs)
            logits = outputs.logits
    
            loss = criterion(logits, labels)
            loss.backward()
            optimizer.step()
            optimizer.zero_grad()
timer_solid_end = time.time()
solid_model.to("cpu")
solid_model.save_pretrained(f'models/{base_model_name}_solid')

  0%|                                                                                        | 0/13334 [00:00<?, ?it/s]C:\Users\mdoga\AppData\Local\Temp\ipykernel_59176\3262206845.py:22: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  labels = torch.tensor(batch['label']).clone().detach().to(device)
100%|████████████████████████████████████████████████████████████████████████████| 13334/13334 [06:35<00:00, 33.69it/s]


In [43]:
## solid + tr_offenseval

solidTr_model = solid_model
solidTr_model.to(device)
solidTr_model.train()
optimizer = optim.AdamW(solidTr_model.parameters(), lr=1e-5)

num_epochs = 1

timer_solidtr_start = time.time()

for epoch in range(num_epochs):

    dataloader_iter = iter(troffenseval_train_loader)
    num_batches = len(troffenseval_train_loader)
    for _ in tqdm(range(num_batches)):
        # Train on labeled data
        batch = next(dataloader_iter, None)
        if batch is not None:
            inputs = tokenizer(batch['text'], padding=True, truncation=True, return_tensors="pt").to(device)
            labels = torch.tensor(batch['label']).clone().detach().to(device)
            outputs = solidTr_model(**inputs)
            logits = outputs.logits
    
            loss = criterion(logits, labels)
            loss.backward()
            optimizer.step()
            optimizer.zero_grad()


timer_solidtr_end = time.time()
solidTr_model.to("cpu")
solidTr_model.save_pretrained(f'models/{base_model_name}_solid_tr')



  0%|                                                                                         | 0/5213 [00:00<?, ?it/s]C:\Users\mdoga\AppData\Local\Temp\ipykernel_59176\3986874014.py:21: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  labels = torch.tensor(batch['label']).clone().detach().to(device)
100%|██████████████████████████████████████████████████████████████████████████████| 5213/5213 [03:54<00:00, 22.25it/s]


In [50]:
## hso
## hso training#

hso_model = baseModel
hso_model.to(device)
hso_model.train()

num_epochs = 1

timer_hso_start = time.time()

for epoch in range(num_epochs):

    dataloader_iter = iter(hso_train_loader)
    num_batches = len(hso_train_loader)
    for _ in tqdm(range(num_batches)):
        # Train on labeled data
        batch = next(dataloader_iter, None)
        if batch is not None:
            inputs = tokenizer(batch['text'], padding=True, truncation=True, return_tensors="pt").to(device)
            labels = torch.tensor(batch['label']).clone().detach().to(device)
            outputs = hso_model(**inputs)
            logits = outputs.logits
    
            loss = criterion(logits, labels)
            loss.backward()
            optimizer.step()
            optimizer.zero_grad()


timer_hso_end = time.time()
hso_model.to("cpu")
hso_model.save_pretrained(f'models/{base_model_name}_hso')




  0%|                                                                                         | 0/5213 [00:00<?, ?it/s]C:\Users\mdoga\AppData\Local\Temp\ipykernel_59176\1187962925.py:21: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  labels = torch.tensor(batch['label']).clone().detach().to(device)
100%|██████████████████████████████████████████████████████████████████████████████| 5213/5213 [03:54<00:00, 22.20it/s]


In [51]:
print("distilbert training times")
print("olid took ", timer_olid_end - timer_olid_start, " seconds to train.")
print("olid + solid took ", timer_olidsolid_end - timer_olidsolid_start, " seconds to train, not including olid training.")
print("solid took ", timer_solid_end - timer_solid_start, " seconds to train.")
print("solid + tr took ", timer_solidtr_end - timer_solidtr_start, " seconds to train.")
print("hso took ", timer_solidtr_end - timer_solidtr_start, " seconds to train.")


distilbert training times
olid took  76.97450399398804  seconds to train.
olid + solid took  400.4078757762909  seconds to train, not including olid training.
solid took  395.81802248954773  seconds to train.
solid + tr took  234.3000774383545  seconds to train.
hso took  234.3000774383545  seconds to train.
